# 5. Regression with a Single Regressor: Hypothesis Tests and Confidence Intervals

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
df = pd.read_excel("data/replication/Replication Data/caschool.xlsx")
df.head()

F:\miniconda3\envs\py\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Observation Number,dist_cod,county,district,gr_span,enrl_tot,teachers,calw_pct,meal_pct,computer,testscr,comp_stu,expn_stu,str,avginc,el_pct,read_scr,math_scr
0,1,75119,Alameda,Sunol Glen Unified,KK-08,195,10.900000,0.510200,2.040800,67,690.799988,0.343590,6384.911133,17.889910,22.690001,0.000000,691.599976,690.000000
1,2,61499,Butte,Manzanita Elementary,KK-08,240,11.150000,15.416700,47.916698,101,661.200012,0.420833,5099.380859,21.524664,9.824000,4.583333,660.500000,661.900024
2,3,61549,Butte,Thermalito Union Elementary,KK-08,1550,82.900002,55.032299,76.322601,169,643.599976,0.109032,5501.954590,18.697226,8.978000,30.000002,636.299988,650.900024
3,4,61457,Butte,Golden Feather Union Elementary,KK-08,243,14.000000,36.475399,77.049202,85,647.700012,0.349794,7101.831055,17.357143,8.978000,0.000000,651.900024,643.500000
4,5,61523,Butte,Palermo Union Elementary,KK-08,1335,71.500000,33.108601,78.427002,171,640.849976,0.128090,5235.987793,18.671329,9.080333,13.857677,641.799988,639.900024


## Testing Hypothesis About One Of The Regression Coefficients

### Two sided Hypothesis concerning $\beta_1$

If taxpayer claims that class size has no effect on test score $\beta_{class size} = 0$

Use Heteroskedasticity-Robust Standard Errors  
$$  \hat{\sigma}^2_{\beta_0} = \frac{1}{n} \times \frac{\frac{1}{n-2} \sum H_i^2\hat{u}^2_i}{{(\frac{1}{n}\sum H_i^2)}^2} $$

$$ H_i^2 = 1 - (\frac{\bar{x}}{\frac{1}{n}\sum X^2_i})X_i$$

In [3]:
model = smf.ols('testscr ~ str', data=df).fit(cov_type='HC2')
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                testscr   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     19.13
Date:                Tue, 09 Jul 2024   Prob (F-statistic):           1.55e-05
Time:                        13:02:21   Log-Likelihood:                -1822.2
No. Observations:                 420   AIC:                             3648.
Df Residuals:                     418   BIC:                             3657.
Df Model:                           1                                         
Covariance Type:                  HC2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    698.9330     10.400     67.206      0.000     678.550     719.316
str           -2.2798      0.521     -4.373      0.000      -3.302      -1.258
==============================================================================
Omnibus:                        5.390   Durbin-Watson:                   0.129
Prob(Omnibus):                  0.068   Jarque-Bera (JB):                3.589
Skew:                          -0.012   Prob(JB):                        0.166
Kurtosis:                       2.548   Cond. No.                         207.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC2)
"""

In [4]:
SER = math.sqrt(model.ssr/(model.nobs-2))
SER

18.58096749678702

$$ t_{act} = -4.38  \text{, p value } =  0.000 $$
null hypothesis is
rejected in favor of the two-sided alternative at the 5% significance level.

### Reporting Regression Equation

\begin{equation}
\text{TestScore} = 698.9 - 2.28 \times \text{STR}
\end{equation}

\begin{equation*}
\begin{array}{r@{}r@{}r}
& &(10.4) &(0.52)
\end{array}
\end{equation*}

\begin{align*}
R^2 &= 0.051 \\
\text{SER} &= 18.6
\end{align*}

### one sided Hypothesis concerning $\beta_1$

In [5]:
# two sided against specific value
hypothesis = 'str = 0'
t_test = model.t_test(hypothesis)
t_test

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -2.2798      0.521     -4.373      0.000      -3.302      -1.258

In [6]:
t_stat = t_test.statistic
t_stat

array([[-4.37340446]])

$$  H_1: \beta_1 < 0 $$

In [7]:
from scipy import stats

degree = model.nobs - 2
t_stat = model.tvalues.iloc[1]
p_value = stats.t.cdf(t_stat, degree)
p_value

np.float64(7.730976280937029e-06)

In [8]:
int(7.730976280937029e-06)

0

reject the null hypthesis,  
you can reject the angry taxpayer’s assertion that the negative estimate of the slope arose purely because of
random sampling variation at the 1% significance level.

## Confidence Intervals for a Regression Coefficient

In [9]:
model.conf_int()

,0,1
Intercept,678.549739,719.316166
str,-3.301516,-1.258100


The value $\beta_1 = 0$ is not contained in this confidence interval,  the hypothesis $\beta_1 = 0$ can be rejected at
the 5% significance level.


### Confidence intervals for predicted effects of changing X.

If  the student teacher ratio is reduced  by 2

In [10]:
model.conf_int()[1:]*-2

,0,1
str,6.603032,2.516201


decreasing the
student–teacher ratio by 2 is estimated to increase test scores by between 2.52 and
6.60 points, with a 95% confidence level.

## Regression When X Is a Binary Variable

suppose you have a variable $D$ that equals either 0 or 1,   
1 if the student–teacher ratio is less than 20,   
0 otherwise

In [11]:
df.columns

Index(['Observation Number', 'dist_cod', 'county', 'district', 'gr_span',
       'enrl_tot', 'teachers', 'calw_pct', 'meal_pct', 'computer', 'testscr',
       'comp_stu', 'expn_stu', 'str', 'avginc', 'el_pct', 'read_scr',
       'math_scr'],
      dtype='object')

In [12]:
df['D'] = (df['str'] < 20).astype(int)
df.head()

,Observation Number,dist_cod,county,district,gr_span,enrl_tot,teachers,calw_pct,meal_pct,computer,testscr,comp_stu,expn_stu,str,avginc,el_pct,read_scr,math_scr,D
0,1,75119,Alameda,Sunol Glen Unified,KK-08,195,10.900000,0.510200,2.040800,67,690.799988,0.343590,6384.911133,17.889910,22.690001,0.000000,691.599976,690.000000,1
1,2,61499,Butte,Manzanita Elementary,KK-08,240,11.150000,15.416700,47.916698,101,661.200012,0.420833,5099.380859,21.524664,9.824000,4.583333,660.500000,661.900024,0
2,3,61549,Butte,Thermalito Union Elementary,KK-08,1550,82.900002,55.032299,76.322601,169,643.599976,0.109032,5501.954590,18.697226,8.978000,30.000002,636.299988,650.900024,1
3,4,61457,Butte,Golden Feather Union Elementary,KK-08,243,14.000000,36.475399,77.049202,85,647.700012,0.349794,7101.831055,17.357143,8.978000,0.000000,651.900024,643.500000,1
4,5,61523,Butte,Palermo Union Elementary,KK-08,1335,71.500000,33.108601,78.427002,171,640.849976,0.128090,5235.987793,18.671329,9.080333,13.857677,641.799988,639.900024,1


In [13]:
model2 = smf.ols('testscr ~ D', data=df).fit(cov_type='HC2')
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                testscr   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     16.34
Date:                Tue, 09 Jul 2024   Prob (F-statistic):           6.29e-05
Time:                        13:02:22   Log-Likelihood:                -1825.4
No. Observations:                 420   AIC:                             3655.
Df Residuals:                     418   BIC:                             3663.
Df Model:                           1                                         
Covariance Type:                  HC2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    649.9788      1.323    491.151      0.000     647.385     652.573
D              7.3724      1.824      4.043      0.000       3.798      10.947
==============================================================================
Omnibus:                        3.061   Durbin-Watson:                   0.110
Prob(Omnibus):                  0.216   Jarque-Bera (JB):                2.448
Skew:                           0.052   Prob(JB):                        0.294
Kurtosis:                       2.641   Cond. No.                         2.81
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC2)
"""

In [14]:
SER = math.sqrt(model2.ssr/(model2.nobs-2))
SER

18.721328714828324

the average test score for the subsample with student–teacher ratios greater than or equal to 20 (
D = 0) is 650.0,  
the average test score for the subsample with student–teacher
ratios less than 20 (D = 1) is 650.0 + 7.4 = 657.4.  
The difference between the sample average test scores for the two groups is 7.4. This is the OLS estimate of $\beta_1$

t statistic for $ \beta_1 $ is 4.04 with p value = 0  
the hypothesis that the population mean test scores in districts with high and low student–teacher
ratios are the same can be rejected at the 5% significance level.

In [15]:
model2.conf_int()

,0,1
Intercept,647.385073,652.572625
D,3.798046,10.946774


 Confidence interval excludes $\beta_1 $ = 0, so that the hypothesis $\beta_1 $ = 0 can be rejected at the 5% significance level.

## Heteroskedasticity and Homoskedasticity

Always use heteroskedasticity-robust standard errors

## Key Concepts

### 1. General Form of the t-Statistic

$$
t = \frac{\text{estimator} - \text{hypothesized value}}{\text{standard error of the estimator}}
$$

### 2. Testing the Hypothesis  $ \beta_1 = \beta_{1,0} $  
### Against the Alternative $ \beta_1 \neq \beta_{1,0} $


1. Compute the standard error of $ \beta_1$ ,  $SE(\beta_1)$  
2. Compute the t-statistic 
3. Compute the p-value $ \text{p-value} = \Pr \left( |Z| > |t_{\text{act}}| \right) = 2\Phi \left( -|t_{\text{act}}| \right)$

Reject the hypothesis at the 5% significance level if the p-value is less than 0.05 or, equivalently, if | $t_{act} $| > 1.96.

### 3. Confidence Interval

A 95% two-sided confidence interval for $ \beta_1 $ is an interval that contains the true
value of $ \beta_1 $ with a 95% probability;   
that is, it contains the true value of $ \beta_1 $ in 95%
of all possible randomly drawn samples.   
Equivalently, it is the set of values of $ \beta_1 $
that cannot be rejected by a 5% two-sided hypothesis test.   
When the sample size
is large, it is constructed as
95% confidence interval for $\beta_1 = \hat{\beta_1} \pm 1.96 \times \text{SE}(\hat{\beta}_1) $

### 4. Heteroskedasticity and Homoskedasticity


The error term $ u_i $  is homoskedastic if the variance of the conditional distribution of $ u_i $ given  $ X_i $   
$ \text{ var}(u_i \mid X_i = x) $, is constant for  $ i = 1, \ldots, n $  and in particular does
not depend on  $ x $.  Otherwise, the error term is heteroskedastic.

## Exercises

🚧 Under Construction